# Consignes

Intégrer le fichier USvideos.csv qui représente un ensemble de 8000 vidéos Youtube. 

Merger le fichier US_category_id.json pour récupérer le nom des catégories. Il conviendra de bien spécifier l'ID du document.


# Questions 
- 1) Récupérer toutes les vidéos de la chaîne Apple.
- 2) Compter le nombre de catégories différentes
- 3) Si vous ne l'avez pas déjà fait, découper les tags en listes et mettre à jour les tags de chacun des documents avec une requête update.
- 4) Récupérer les vidéos les plus vues.
- 5) Compter le nombre moyen de vues en fonction de la catégorie.
- 6) Récupérer les chaines Youtube avec la plus grande moyenne de likes.

In [1]:
import pandas as pd
import pymongo

In [2]:
client = pymongo.MongoClient()
database = client['exercices']
collection = database['youtube']

In [3]:
# Configurer pandas pour afficher toutes les colonnes
pd.set_option('display.max_columns', None)

df_youtube = pd.read_csv("./data/USvideos.csv")
print(f"Nombre de vidéos: {len(df_youtube)}")
print(f"Nombre de colonnes: {len(df_youtube.columns)}")
df_youtube.head(2)

Nombre de vidéos: 7992
Nombre de colonnes: 11


,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,13.09


## Question 0

### Netoyer les données

In [4]:
# Vérifier les valeurs manquantes et les types
print("Valeurs manquantes par colonne:")
print(df_youtube.isnull().sum())
print("\n")
df_youtube.info()

Valeurs manquantes par colonne:
video_id          0
title             0
channel_title     0
category_id       0
tags              0
views             0
likes             0
dislikes          0
comment_total     0
thumbnail_link    0
date              0
dtype: int64


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7992 entries, 0 to 7991
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   video_id        7992 non-null   object 
 1   title           7992 non-null   object 
 2   channel_title   7992 non-null   object 
 3   category_id     7992 non-null   int64  
 4   tags            7992 non-null   object 
 5   views           7992 non-null   int64  
 6   likes           7992 non-null   int64  
 7   dislikes        7992 non-null   int64  
 8   comment_total   7992 non-null   int64  
 9   thumbnail_link  7992 non-null   object 
 10  date            7992 non-null   float64
dtypes: float64(1), int64(5), object(5)
m

### Importer les données

In [5]:
# Charger les catégories depuis le fichier JSON
import json

with open('./data/US_category_id.json', 'r') as f:
    categories_data = json.load(f)

# Créer un dictionnaire de mapping category_id -> category_name
category_mapping = {}
for item in categories_data['items']:
    category_mapping[int(item['id'])] = item['snippet']['title']

print("Catégories disponibles:")
for cat_id, cat_name in category_mapping.items():
    print(f"  {cat_id}: {cat_name}")

# Ajouter le nom de la catégorie au DataFrame
df_youtube['category_name'] = df_youtube['category_id'].map(category_mapping)

# Vérifier
print(f"\nAprès merge: {len(df_youtube.columns)} colonnes")
print(df_youtube[['category_id', 'category_name']].head())

Catégories disponibles:
  1: Film & Animation
  2: Autos & Vehicles
  10: Music
  15: Pets & Animals
  17: Sports
  18: Short Movies
  19: Travel & Events
  20: Gaming
  21: Videoblogging
  22: People & Blogs
  23: Comedy
  24: Entertainment
  25: News & Politics
  26: Howto & Style
  27: Education
  28: Science & Technology
  29: Nonprofits & Activism
  30: Movies
  31: Anime/Animation
  32: Action/Adventure
  33: Classics
  34: Comedy
  35: Documentary
  36: Drama
  37: Family
  38: Foreign
  39: Horror
  40: Sci-Fi/Fantasy
  41: Thriller
  42: Shorts
  43: Shows
  44: Trailers

Après merge: 12 colonnes
   category_id         category_name
0           24         Entertainment
1           28  Science & Technology
2           22        People & Blogs
3           28  Science & Technology
4           23                Comedy


In [10]:
# Nettoyer et importer dans MongoDB
# Supprimer les anciennes données si elles existent
collection.delete_many({})
print("Collection vidée")

# Supprimer les doublons basés sur video_id (garder la première occurrence)
df_youtube_unique = df_youtube.drop_duplicates(subset='video_id', keep='first')
print(f"Doublons supprimés: {len(df_youtube)} -> {len(df_youtube_unique)} vidéos uniques")

# Utiliser video_id comme _id pour MongoDB
df_youtube_unique['_id'] = df_youtube_unique['video_id']

# Importer dans MongoDB (sans conversion de date - on garde le format original)
records = df_youtube_unique.to_dict('records')
collection.insert_many(records)

print(f"{len(records)} vidéos importées dans MongoDB")
print(f"{len(df_youtube.columns)} colonnes par document")

# Vérifier
sample = collection.find_one()
print(f"\nChamps dans MongoDB: {list(sample.keys())}")


Collection vidée
Doublons supprimés: 7992 -> 2364 vidéos uniques
2364 vidéos importées dans MongoDB
13 colonnes par document

Champs dans MongoDB: ['_id', 'video_id', 'title', 'channel_title', 'category_id', 'tags', 'views', 'likes', 'dislikes', 'comment_total', 'thumbnail_link', 'date', 'category_name']


C:\Users\icarr\AppData\Local\Temp\ipykernel_23200\2504023646.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_youtube_unique['_id'] = df_youtube_unique['video_id']


## Question 1  

In [11]:
# 1) Récupérer toutes les vidéos de la chaîne Apple
apple_videos = list(collection.find({"channel_title": "Apple"}))

print(f"Nombre de vidéos de la chaîne Apple: {len(apple_videos)}")
print("\nExemples de vidéos Apple:")
for i, video in enumerate(apple_videos[:3], 1):
    print(f"\n{i}. {video['title']}")
    print(f"   Vues: {video['views']:,}")
    print(f"   Likes: {video['likes']:,}")

Nombre de vidéos de la chaîne Apple: 3

Exemples de vidéos Apple:

1. iPhone X — Introducing iPhone X — Apple
   Vues: 7,860,119
   Likes: 185,853

2. Apple Watch Series 3 + Apple Music — Roll — Apple
   Vues: 782,543
   Likes: 14,647

3. iPhone 8 Plus — Portraits of Her  — Apple
   Vues: 117,564
   Likes: 1,240


## Question 2

In [12]:
# 2) Compter le nombre de catégories différentes
categories = collection.distinct("category_name")

print(f"Nombre de catégories différentes: {len(categories)}")
print("\nListe des catégories:")
for cat in sorted(categories):
    count = collection.count_documents({"category_name": cat})
    print(f"  - {cat}: {count} vidéos")

Nombre de catégories différentes: 16

Liste des catégories:
  - Autos & Vehicles: 37 vidéos
  - Comedy: 213 vidéos
  - Education: 94 vidéos
  - Entertainment: 477 vidéos
  - Film & Animation: 101 vidéos
  - Gaming: 29 vidéos
  - Howto & Style: 262 vidéos
  - Music: 335 vidéos
  - News & Politics: 198 vidéos
  - Nonprofits & Activism: 5 vidéos
  - People & Blogs: 257 vidéos
  - Pets & Animals: 37 vidéos
  - Science & Technology: 151 vidéos
  - Shows: 1 vidéos
  - Sports: 149 vidéos
  - Travel & Events: 18 vidéos


## Question 3

In [14]:
# 3) Découper les tags en listes et mettre à jour
# Vérifier d'abord le format actuel des tags
sample_video = collection.find_one()
print(f"Format actuel des tags: {type(sample_video.get('tags'))}")
print(f"Exemple: {sample_video.get('tags')[:100]}...")

# Si les tags sont une chaîne avec des pipes "|", les découper
if isinstance(sample_video.get('tags'), str):
    # Mettre à jour tous les documents
    videos_with_tags = collection.find({"tags": {"$exists": True, "$ne": "[none]"}})
    
    updated_count = 0
    for video in videos_with_tags:
        if isinstance(video['tags'], str) and video['tags'] != "[none]":
            # Découper les tags
            tags_list = [tag.strip() for tag in video['tags'].split('|') if tag.strip()]
            
            # Mettre à jour le document
            collection.update_one(
                {"_id": video['_id']},
                {"$set": {"tags": tags_list}}
            )
            updated_count += 1
    
    print(f"\n{updated_count} vidéos mises à jour avec tags en liste")
    
    # Vérifier
    sample_updated = collection.find_one({"tags": {"$type": "array"}})
    if sample_updated:
        print(f"\nExemple après mise à jour:")
        print(f"Tags: {sample_updated.get('tags')[:5]}...")
else:
    print("Les tags sont déjà au format liste !")

Format actuel des tags: <class 'list'>
Exemple: ['logan paul vlog', 'logan paul', 'logan', 'paul', 'olympics', 'logan paul youtube', 'vlog', 'daily', 'comedy', 'hollywood', 'parrot', 'maverick', 'bird', 'maverick clothes', 'diamond play button', 'logan paul diamond play button', '10M subscribers', 'logan paul 1 year vlogging', '1 year vlog', 'dwarf mamba play button', 'logan paul history', 'youtube history', '10M', '10M plaque', 'youtube button', 'diamond button', 'logang', 'logang 4 life']...
Les tags sont déjà au format liste !


## Question 4

In [15]:
# 4) Récupérer les vidéos les plus vues
top_videos = list(collection.find().sort("views", -1).limit(10))

print("="*70)
print("Top 10 des vidéos les plus vues")
print("="*70)

for i, video in enumerate(top_videos, 1):
    print(f"\n{i}. {video['title']}")
    print(f"   Chaîne: {video['channel_title']}")
    print(f"   Vues: {video['views']:,}")
    print(f"   Likes: {video['likes']:,}")
    print(f"   Catégorie: {video.get('category_name', 'N/A')}")

Top 10 des vidéos les plus vues

1. ZAYN - Dusk Till Dawn ft. Sia
   Chaîne: ZaynVEVO
   Vues: 36,323,498
   Likes: 1,431,683
   Catégorie: Music

2. BTS (방탄소년단) 'DNA' Official MV
   Chaîne: ibighit
   Vues: 20,565,649
   Likes: 1,478,119
   Catégorie: Music

3. Primitive Technology: Mud Bricks
   Chaîne: Primitive Technology
   Vues: 14,068,244
   Likes: 328,200
   Catégorie: People & Blogs

4. Live footage as Hurricane Irma destroys Maho Beach Cam in St Maarten   9/6/2017
   Chaîne: PTZtv
   Vues: 13,752,002
   Likes: 46,101
   Catégorie: News & Politics

5. Sam Smith - Too Good At Goodbyes (Official Audio)
   Chaîne: SamSmithWorldVEVO
   Vues: 13,728,070
   Likes: 468,636
   Catégorie: Music

6. Nerf Bow Trick Shots | Dude Perfect
   Chaîne: Dude Perfect
   Vues: 10,535,242
   Likes: 384,920
   Catégorie: Sports

7. Sneaky toddler steals Prince Harry's popcorn
   Chaîne: The Royal Family Channel
   Vues: 10,431,296
   Likes: 31,706
   Catégorie: News & Politics

8. Logan Paul - Outt

## Question 5

In [16]:
# 5) Compter le nombre moyen de vues en fonction de la catégorie
pipeline = [
    {
        "$group": {
            "_id": "$category_name",
            "avg_views": {"$avg": "$views"},
            "total_videos": {"$sum": 1}
        }
    },
    {
        "$sort": {"avg_views": -1}
    }
]

results = list(collection.aggregate(pipeline))

print("Nombre moyen de vues par catégorie:")
print("="*60)
for result in results:
    print(f"{result['_id']:20s} : {result['avg_views']:12,.0f} vues (sur {result['total_videos']} vidéos)")

Nombre moyen de vues par catégorie:
Comedy               :      828,485 vues (sur 213 vidéos)
Music                :      697,599 vues (sur 335 vidéos)
People & Blogs       :      597,891 vues (sur 257 vidéos)
Nonprofits & Activism :      572,023 vues (sur 5 vidéos)
Film & Animation     :      558,289 vues (sur 101 vidéos)
Entertainment        :      555,071 vues (sur 477 vidéos)
Science & Technology :      515,854 vues (sur 151 vidéos)
Gaming               :      492,390 vues (sur 29 vidéos)
Sports               :      470,686 vues (sur 149 vidéos)
Autos & Vehicles     :      415,129 vues (sur 37 vidéos)
Pets & Animals       :      412,382 vues (sur 37 vidéos)
Howto & Style        :      405,748 vues (sur 262 vidéos)
News & Politics      :      374,883 vues (sur 198 vidéos)
Education            :      368,229 vues (sur 94 vidéos)
Travel & Events      :      300,664 vues (sur 18 vidéos)
Shows                :        8,259 vues (sur 1 vidéos)


## Question 6 

In [17]:
# 6) Récupérer les chaînes Youtube avec la plus grande moyenne de likes
pipeline = [
    {
        "$group": {
            "_id": "$channel_title",
            "avg_likes": {"$avg": "$likes"},
            "total_videos": {"$sum": 1},
            "total_views": {"$sum": "$views"}
        }
    },
    {
        "$sort": {"avg_likes": -1}
    },
    {
        "$limit": 15
    }
]

results = list(collection.aggregate(pipeline))

print("Top 15 des chaînes avec la plus grande moyenne de likes:")
print("="*80)
for i, result in enumerate(results, 1):
    print(f"\n{i}. {result['_id']}")
    print(f"   Moyenne de likes: {result['avg_likes']:,.0f}")
    print(f"   Nombre de vidéos: {result['total_videos']}")
    print(f"   Total de vues: {result['total_views']:,.0f}")

Top 15 des chaînes avec la plus grande moyenne de likes:

1. ZaynVEVO
   Moyenne de likes: 1,431,683
   Nombre de vidéos: 1
   Total de vues: 36,323,498

2. ibighit
   Moyenne de likes: 1,112,071
   Nombre de vidéos: 2
   Total de vues: 27,136,260

3. melanie martinez
   Moyenne de likes: 902,166
   Nombre de vidéos: 1
   Total de vues: 6,545,187

4. jypentertainment
   Moyenne de likes: 721,796
   Nombre de vidéos: 1
   Total de vues: 6,530,523

5. Logan Paul Vlogs
   Moyenne de likes: 601,534
   Nombre de vidéos: 2
   Total de vues: 13,667,504

6. Wengie
   Moyenne de likes: 455,904
   Nombre de vidéos: 1
   Total de vues: 4,632,889

7. Conor Maynard
   Moyenne de likes: 389,273
   Nombre de vidéos: 1
   Total de vues: 5,326,424

8. Dude Perfect
   Moyenne de likes: 363,320
   Nombre de vidéos: 2
   Total de vues: 19,331,053

9. PewDiePie
   Moyenne de likes: 351,994
   Nombre de vidéos: 2
   Total de vues: 7,558,485

10. Primitive Technology
   Moyenne de likes: 328,200
   Nombre de